In [ ]:
# https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html#sphx-glr-auto-examples-tutorials-run-lda-py
# https://radimrehurek.com/topic_modeling_tutorial/2%20-%20Topic%20Modeling.html
# https://radimrehurek.com/gensim/models/ldamodel.html

# https://github.com/DocNow/twarc

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
tweets = pd.read_csv('tweets_no_dupes_v2.csv',index_col = 0, low_memory = False)

In [3]:
tweets.head()

,tweet_created_at,tweet_id,tweet_id_str,tweet_text,tweet_truncated,tweet_source,tweet_in_reply_to_status_id,tweet_in_reply_to_status_id_str,tweet_in_reply_to_user_id,tweet_in_reply_to_user_id_str,...,tweet_place_contained_within,tweet_place_bounding_box_type,tweet_place_bounding_box_coordinates,tweet_geo_type,tweet_geo_coordinates,tweet_coordinates_type,tweet_coordinates_coordinates,tweet_lang,tweet_full_text,search_tweets
0,Fri Oct 30 23:48:05 +0000 2020,1322324427653132288,1322324427653132288,"Nigerians are so over COVID, they don’t even b...",False,"<a href=""http://twitter.com/download/iphone"" r...",None,None,None,None,...,NaN,Polygon,"[[[3.402322, 6.396264], [3.402322, 6.486797], ...",NaN,NaN,NaN,NaN,en,NaN,Covid
1,Fri Oct 30 23:08:10 +0000 2020,1322314381926010880,1322314381926010880,Nigerians no even let covid settle. Is it here...,False,"<a href=""http://twitter.com/download/android"" ...",None,None,None,None,...,NaN,Polygon,"[[[3.402322, 6.396264], [3.402322, 6.486797], ...",NaN,NaN,NaN,NaN,en,NaN,Covid
2,Fri Oct 30 22:44:29 +0000 2020,1322308420842770433,1322308420842770433,Covid was on holiday sha!!!! Awon alase https:...,False,"<a href=""http://twitter.com/download/iphone"" r...",None,None,None,None,...,NaN,Polygon,"[[[3.26291, 6.639331], [3.26291, 6.740074], [3...",NaN,NaN,NaN,NaN,en,NaN,Covid
3,Fri Oct 30 22:43:42 +0000 2020,1322308224654151681,1322308224654151681,Covid-19 is backBe careful guys. Use your mask...,False,"<a href=""http://twitter.com/download/android"" ...",None,None,None,None,...,NaN,Polygon,"[[[3.198942, 6.482676], [3.198942, 6.698546], ...",NaN,NaN,NaN,NaN,en,NaN,Covid
4,Fri Oct 30 22:38:25 +0000 2020,1322306894581649415,1322306894581649415,COVId in Lagos - quite worrisome https://t.co/...,False,"<a href=""http://twitter.com/download/iphone"" r...",None,None,None,None,...,NaN,Polygon,"[[[3.198942, 6.482676], [3.198942, 6.698546], ...",NaN,NaN,NaN,NaN,en,NaN,Covid


In [17]:
tweets['tweet_created_at'] = pd.to_datetime(tweets['tweet_created_at'])

In [18]:
start_date = '2020-01-01'
end_date = '2020-04-01'

mask_1 = (tweets['tweet_created_at'] >= start_date) & (tweets['tweet_created_at'] < end_date)

In [20]:
temp_tweets_1 = tweets.loc[mask_1]

In [21]:
temp_tweets_1.head()

,tweet_created_at,tweet_id,tweet_id_str,tweet_text,tweet_truncated,tweet_source,tweet_in_reply_to_status_id,tweet_in_reply_to_status_id_str,tweet_in_reply_to_user_id,tweet_in_reply_to_user_id_str,...,tweet_place_contained_within,tweet_place_bounding_box_type,tweet_place_bounding_box_coordinates,tweet_geo_type,tweet_geo_coordinates,tweet_coordinates_type,tweet_coordinates_coordinates,tweet_lang,tweet_full_text,search_tweets
48206,2020-03-31 23:56:13+00:00,1245137856328347649,1245137856328347649,#StayAtHome Is it really difficult to comply w...,True,"<a href=""http://twitter.com/download/android"" ...",None,None,None,None,...,NaN,Polygon,"[[[3.402322, 6.396264], [3.402322, 6.486797], ...",NaN,NaN,NaN,NaN,en,#StayAtHome Is it really difficult to comply w...,Covid
48207,2020-03-31 23:53:11+00:00,1245137094550355974,1245137094550355974,@smile_poc @ajeromorgan Candles for lost ones ...,False,"<a href=""http://twitter.com/download/iphone"" r...",1245136648225972224,1245136648225972224,236855884,236855884,...,NaN,Polygon,"[[[3.198942, 6.482676], [3.198942, 6.698546], ...",NaN,NaN,NaN,NaN,en,NaN,Covid
48208,2020-03-31 23:51:44+00:00,1245136730380021760,1245136730380021760,Beware of IAD illness Anxiety Disorder. Hypoch...,True,"<a href=""http://twitter.com/download/iphone"" r...",None,None,None,None,...,NaN,Polygon,"[[[7.304554, 8.961887], [7.304554, 9.185441], ...",NaN,NaN,NaN,NaN,en,Beware of IAD illness Anxiety Disorder. Hypoch...,Covid
48209,2020-03-31 23:51:39+00:00,1245136706480885761,1245136706480885761,we win the fight against COVID-19Thank you. S...,True,"<a href=""http://twitter.com/download/iphone"" r...",1245136704580792322,1245136704580792322,328430837,328430837,...,NaN,Polygon,"[[[6.93641, 7.769396], [6.93641, 9.401598], [9...",NaN,NaN,NaN,NaN,en,we win the fight against COVID-19Thank you. S...,Covid
48210,2020-03-31 23:51:38+00:00,1245136704580792322,1245136704580792322,"NASSA therefore, call on the general public to...",True,"<a href=""http://twitter.com/download/iphone"" r...",1245136702651478016,1245136702651478016,328430837,328430837,...,NaN,Polygon,"[[[6.93641, 7.769396], [6.93641, 9.401598], [9...",NaN,NaN,NaN,NaN,en,"NASSA therefore, call on the general public to...",Covid


In [22]:
start_date = '2020-04-01'
end_date = '2020-07-01'

mask_2 = (tweets['tweet_created_at'] >= start_date) & (tweets['tweet_created_at'] < end_date)

In [23]:
temp_tweets_2 = tweets.loc[mask_2]
temp_tweets_2.head()

,tweet_created_at,tweet_id,tweet_id_str,tweet_text,tweet_truncated,tweet_source,tweet_in_reply_to_status_id,tweet_in_reply_to_status_id_str,tweet_in_reply_to_user_id,tweet_in_reply_to_user_id_str,...,tweet_place_contained_within,tweet_place_bounding_box_type,tweet_place_bounding_box_coordinates,tweet_geo_type,tweet_geo_coordinates,tweet_coordinates_type,tweet_coordinates_coordinates,tweet_lang,tweet_full_text,search_tweets
11731,2020-06-30 23:28:19+00:00,1278108130988228609,1278108130988228609,"Guy covid_19 is real please stay Safe,Happy Ne...",False,"<a href=""http://twitter.com/download/android"" ...",None,None,None,None,...,NaN,Polygon,"[[[5.376385, 4.269669], [5.376385, 5.416369], ...",NaN,NaN,NaN,NaN,en,NaN,Covid
11732,2020-06-30 23:19:28+00:00,1278105904198623234,1278105904198623234,"Astagfirullah, I know covid is real but this o...",False,"<a href=""http://twitter.com/download/android"" ...",None,None,None,None,...,NaN,Polygon,"[[[8.447599, 11.929633], [8.447599, 12.080367]...",NaN,NaN,NaN,NaN,en,NaN,Covid
11733,2020-06-30 23:07:14+00:00,1278102827450105857,1278102827450105857,All sickness na Covid-19 for Naija,False,"<a href=""http://twitter.com/download/android"" ...",None,None,None,None,...,NaN,Polygon,"[[[3.213147, 6.427886], [3.213147, 6.558339], ...",NaN,NaN,NaN,NaN,en,NaN,Covid
11734,2020-06-30 23:07:00+00:00,1278102769178742784,1278102769178742784,COVID 19 UPDATE IN KOGI STATE: THE NEED FOR CO...,True,"<a href=""http://twitter.com/download/android"" ...",None,None,None,None,...,NaN,Polygon,"[[[6.691274, 7.739657], [6.691274, 7.826825], ...",NaN,NaN,NaN,NaN,en,COVID 19 UPDATE IN KOGI STATE: THE NEED FOR CO...,Covid
11735,2020-06-30 22:23:57+00:00,1278091935631147010,1278091935631147010,"While the world battles COVID-19, here comes a...",True,"<a href=""http://twitter.com/download/android"" ...",None,None,None,None,...,NaN,Polygon,"[[[3.568777, 6.469079], [3.568777, 6.469079], ...",NaN,NaN,NaN,NaN,en,"While the world battles COVID-19, here comes a...",Covid


In [24]:
start_date = '2020-07-01'
end_date = '2020-12-31'

mask_3 = (tweets['tweet_created_at'] >= start_date) & (tweets['tweet_created_at'] <= end_date)

In [25]:
temp_tweets_3 = tweets.loc[mask_3]
temp_tweets_3.head()

,tweet_created_at,tweet_id,tweet_id_str,tweet_text,tweet_truncated,tweet_source,tweet_in_reply_to_status_id,tweet_in_reply_to_status_id_str,tweet_in_reply_to_user_id,tweet_in_reply_to_user_id_str,...,tweet_place_contained_within,tweet_place_bounding_box_type,tweet_place_bounding_box_coordinates,tweet_geo_type,tweet_geo_coordinates,tweet_coordinates_type,tweet_coordinates_coordinates,tweet_lang,tweet_full_text,search_tweets
0,2020-10-30 23:48:05+00:00,1322324427653132288,1322324427653132288,"Nigerians are so over COVID, they don’t even b...",False,"<a href=""http://twitter.com/download/iphone"" r...",None,None,None,None,...,NaN,Polygon,"[[[3.402322, 6.396264], [3.402322, 6.486797], ...",NaN,NaN,NaN,NaN,en,NaN,Covid
1,2020-10-30 23:08:10+00:00,1322314381926010880,1322314381926010880,Nigerians no even let covid settle. Is it here...,False,"<a href=""http://twitter.com/download/android"" ...",None,None,None,None,...,NaN,Polygon,"[[[3.402322, 6.396264], [3.402322, 6.486797], ...",NaN,NaN,NaN,NaN,en,NaN,Covid
2,2020-10-30 22:44:29+00:00,1322308420842770433,1322308420842770433,Covid was on holiday sha!!!! Awon alase https:...,False,"<a href=""http://twitter.com/download/iphone"" r...",None,None,None,None,...,NaN,Polygon,"[[[3.26291, 6.639331], [3.26291, 6.740074], [3...",NaN,NaN,NaN,NaN,en,NaN,Covid
3,2020-10-30 22:43:42+00:00,1322308224654151681,1322308224654151681,Covid-19 is backBe careful guys. Use your mask...,False,"<a href=""http://twitter.com/download/android"" ...",None,None,None,None,...,NaN,Polygon,"[[[3.198942, 6.482676], [3.198942, 6.698546], ...",NaN,NaN,NaN,NaN,en,NaN,Covid
4,2020-10-30 22:38:25+00:00,1322306894581649415,1322306894581649415,COVId in Lagos - quite worrisome https://t.co/...,False,"<a href=""http://twitter.com/download/iphone"" r...",None,None,None,None,...,NaN,Polygon,"[[[3.198942, 6.482676], [3.198942, 6.698546], ...",NaN,NaN,NaN,NaN,en,NaN,Covid


In [3]:
tweets['tweet_text'] = np.where((tweets.tweet_truncated == 'True'),tweets.tweet_full_text , tweets.tweet_text)

NameError: name 'np' is not defined

In [4]:
tweets['tweet_text'].tolist()[:5]

['Nigerians are so over COVID, they don’t even bother to wear masks anymore.',
 'Nigerians no even let covid settle. Is it here, is it there until we end am.',
 'Covid was on holiday sha!!!! Awon alase https://t.co/WesAOJK1ls',
 'Covid-19 is backBe careful guys. Use your masks, sanitize and wash your hands regular, and maintain social distancing',
 'COVId in Lagos - quite worrisome https://t.co/pS0lhWF9ZP']

In [5]:
tweets_txt = tweets['tweet_text'].tolist()

In [6]:
def remove_url(text):
    txt = re.sub(r'http\S+', '', text)
    return txt

In [21]:
tweets_txt_sans_url = [remove_url(text) for text in tweets_txt]

In [22]:
len(tweets_txt_sans_url)

185661

In [23]:
import nltk

In [24]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [26]:
# Tokenize the documents.
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(tweets_txt_sans_url)):
    tweets_txt_sans_url[idx] = tweets_txt_sans_url[idx].lower()  # Convert to lowercase.
    tweets_txt_sans_url[idx] = tokenizer.tokenize(tweets_txt_sans_url[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in tweet if not token.isnumeric()] for tweet in tweets_txt_sans_url]

# Remove words that are only one character.
docs = [[token for token in tweet if len(token) > 1] for tweet in docs]

# Remove stopwords
docs = [[token for token in tweet if token not in stop_words] for tweet in docs]

In [27]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

INFO : collecting all words and their counts
INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO : PROGRESS: at sentence #10000, processed 154117 words and 128166 word types
INFO : PROGRESS: at sentence #20000, processed 311254 words and 234836 word types
INFO : PROGRESS: at sentence #30000, processed 468965 words and 335688 word types
INFO : PROGRESS: at sentence #40000, processed 627368 words and 428408 word types
INFO : PROGRESS: at sentence #50000, processed 783061 words and 516097 word types
INFO : PROGRESS: at sentence #60000, processed 933822 words and 596788 word types
INFO : PROGRESS: at sentence #70000, processed 1105731 words and 693466 word types
INFO : PROGRESS: at sentence #80000, processed 1269631 words and 785427 word types
INFO : PROGRESS: at sentence #90000, processed 1432464 words and 866158 word types
INFO : PROGRESS: at sentence #100000, processed 1587851 words and 937900 word types
INFO : PROGRESS: at sentence #110000, processed 1708132 words 

In [28]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)

INFO : adding document #0 to Dictionary(0 unique tokens: [])
INFO : adding document #10000 to Dictionary(25133 unique tokens: ['anymore', 'bother', 'covid', 'even', 'masks']...)
INFO : adding document #20000 to Dictionary(38891 unique tokens: ['anymore', 'bother', 'covid', 'even', 'masks']...)
INFO : adding document #30000 to Dictionary(50204 unique tokens: ['anymore', 'bother', 'covid', 'even', 'masks']...)
INFO : adding document #40000 to Dictionary(59531 unique tokens: ['anymore', 'bother', 'covid', 'even', 'masks']...)
INFO : adding document #50000 to Dictionary(68202 unique tokens: ['anymore', 'bother', 'covid', 'even', 'masks']...)
INFO : adding document #60000 to Dictionary(75565 unique tokens: ['anymore', 'bother', 'covid', 'even', 'masks']...)
INFO : adding document #70000 to Dictionary(86902 unique tokens: ['anymore', 'bother', 'covid', 'even', 'masks']...)
INFO : adding document #80000 to Dictionary(95939 unique tokens: ['anymore', 'bother', 'covid', 'even', 'masks']...)
INF

In [29]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [30]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 13369
Number of documents: 185661


In [15]:
import logging
import itertools

logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO  # ipython sometimes messes up the logging setup; restore

def head(stream, n=10):
    """Convenience fnc: return the first `n` elements of the stream, as plain list."""
    return list(itertools.islice(stream, n))

In [32]:
# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 10
chunksize = 10000
passes = 10
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

INFO : using autotuned alpha, starting with [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
INFO : using serial LDA version on this node
INFO : running online (multi-pass) LDA training, 10 topics, 10 passes over the supplied corpus of 185661 documents, updating model once every 10000 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
INFO : PROGRESS: pass 0, at document #10000/185661
INFO : optimized alpha [0.071741104, 0.07470258, 0.079301536, 0.07450368, 0.07297543, 0.09062151, 0.08049784, 0.08106586, 0.08306661, 0.07862542]
INFO : merging changes from 10000 documents into a model of 185661 documents
INFO : topic #0 (0.072): 0.054*"covid" + 0.011*"people" + 0.007*"state" + 0.007*"government" + 0.006*"covid_19" + 0.006*"pandemic" + 0.005*"us" + 0.005*"would" + 0.005*"amp" + 0.004*"nigeria"
INFO : topic #4 (0.073): 0.057*"covid" + 0.009*"us" + 0.008*"covid_19" + 0.008*"pandemic" + 0.007*"one" + 0.006*"government" + 0.006*"go" 

In [33]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

INFO : CorpusAccumulator accumulated stats from 1000 documents
INFO : CorpusAccumulator accumulated stats from 2000 documents
INFO : CorpusAccumulator accumulated stats from 3000 documents
INFO : CorpusAccumulator accumulated stats from 4000 documents
INFO : CorpusAccumulator accumulated stats from 5000 documents
INFO : CorpusAccumulator accumulated stats from 6000 documents
INFO : CorpusAccumulator accumulated stats from 7000 documents
INFO : CorpusAccumulator accumulated stats from 8000 documents
INFO : CorpusAccumulator accumulated stats from 9000 documents
INFO : CorpusAccumulator accumulated stats from 10000 documents
INFO : CorpusAccumulator accumulated stats from 11000 documents
INFO : CorpusAccumulator accumulated stats from 12000 documents
INFO : CorpusAccumulator accumulated stats from 13000 documents
INFO : CorpusAccumulator accumulated stats from 14000 documents
INFO : CorpusAccumulator accumulated stats from 15000 documents
INFO : CorpusAccumulator accumulated stats from 1

Average topic coherence: -5.5530.
[([(0.037206367, 'lockdown'),
   (0.016181761, 'people'),
   (0.015032359, 'covid19'),
   (0.011153676, 'covid'),
   (0.009775944, 'please'),
   (0.0095550455, 'us'),
   (0.009211638, 'nigeria'),
   (0.008934135, 'like'),
   (0.008801816, 'one'),
   (0.008332094, 'ncdc'),
   (0.007064883, 'know'),
   (0.006884901, 'time'),
   (0.006570821, 'need'),
   (0.0065414184, 'get'),
   (0.00643652, 'still'),
   (0.0062726126, 'see'),
   (0.0057873083, 'let'),
   (0.0056259558, 'lagos'),
   (0.005483382, 'even'),
   (0.0049955454, 'virus')],
  -2.9906625516033416),
 ([(0.065167814, 'cases'),
   (0.040025104, 'new'),
   (0.023501217, 'case'),
   (0.022852555, 'kano'),
   (0.021176642, 'lagos'),
   (0.020590123, 'nigeria'),
   (0.019656153, 'test'),
   (0.018285975, 'confirmed'),
   (0.0176242, 'ncdc'),
   (0.017014375, 'testing'),
   (0.016691938, 'covid19'),
   (0.015502953, 'positive'),
   (0.015253876, 'tested'),
   (0.014891805, 'total'),
   (0.013301205, 'nu